In [5]:
import csv
import pickle
import pandas as pd
import numpy as np

In [6]:
from google.cloud import storage

In [7]:
document_data = pd.read_csv(
    "../court_data/2022/documents.csv", 
    on_bad_lines='skip', 
    engine='python', 
    sep="	",
) #.fillna(method="ffill")
document_data.head(10)

,doc_id,court_code,judgment_code,justice_kind,category_code,cause_num,adjudication_date,receipt_date,judge,doc_url,status,date_publ
0,102383845,503.0,5.0,2.0,40467.0,219/14855/21,2021-12-30 00:00:00+02,2022-01-02 00:00:00+02,Медінцева Н. М.,NaN,0.0,2022-01-02 00:00:00+02
1,102391042,4856.0,5.0,4.0,40105.0,560/6776/21,2021-12-29 00:00:00+02,2022-01-02 00:00:00+02,Матохнюк Д.Б.,http://od.reyestr.court.gov.ua/files/51/0c9097...,1.0,2022-01-04 00:00:00+02
2,102390645,4824.0,5.0,1.0,NaN,824/245/21,2021-12-22 00:00:00+02,2022-01-02 00:00:00+02,Нежура Вадим Анатолійович,http://od.reyestr.court.gov.ua/files/51/a53df6...,1.0,2022-01-04 00:00:00+02
3,102391822,2670.0,5.0,4.0,40114.0,640/4558/19,2021-11-02 00:00:00+02,2022-01-02 00:00:00+02,Клименчук Н.М.,http://od.reyestr.court.gov.ua/files/51/904bf7...,1.0,2022-01-04 00:00:00+02
4,102391796,2670.0,3.0,4.0,40110.0,640/10043/21,2021-12-07 00:00:00+02,2022-01-02 00:00:00+02,Клименчук Н.М.,http://od.reyestr.court.gov.ua/files/51/4c9376...,1.0,2022-01-04 00:00:00+02
5,102391789,2670.0,3.0,4.0,40110.0,640/18672/21,2021-12-01 00:00:00+02,2022-01-02 00:00:00+02,Клименчук Н.М.,http://od.reyestr.court.gov.ua/files/51/f1fc47...,1.0,2022-01-04 00:00:00+02
6,102391769,2670.0,3.0,4.0,40088.0,640/3619/19,2021-11-10 00:00:00+02,2022-01-02 00:00:00+02,Клименчук Н.М.,http://od.reyestr.court.gov.ua/files/51/98d4ee...,1.0,2022-01-04 00:00:00+02
7,102391764,2670.0,3.0,4.0,40114.0,640/4230/19,2021-11-05 00:00:00+02,2022-01-02 00:00:00+02,Клименчук Н.М.,http://od.reyestr.court.gov.ua/files/51/dbaa6e...,1.0,2022-01-04 00:00:00+02
8,102391757,2670.0,3.0,4.0,40077.0,640/1726/20,2021-11-02 00:00:00+02,2022-01-02 00:00:00+02,Клименчук Н.М.,http://od.reyestr.court.gov.ua/files/51/145f06...,1.0,2022-01-04 00:00:00+02
9,102388442,1570.0,3.0,4.0,40074.0,420/20761/21,2021-12-30 00:00:00+02,2022-01-02 00:00:00+02,Потоцька Н.В.,http://od.reyestr.court.gov.ua/files/51/5514da...,1.0,2022-01-04 00:00:00+02


Transform data 

In [8]:
document_data.shape

(544914, 12)

In [9]:
administrative_document_data = document_data[document_data["justice_kind"] == 4]
administrative_document_data = administrative_document_data[administrative_document_data["doc_url"].notna()]
administrative_document_data.head(10)

,doc_id,court_code,judgment_code,justice_kind,category_code,cause_num,adjudication_date,receipt_date,judge,doc_url,status,date_publ
1,102391042,4856.0,5.0,4.0,40105.0,560/6776/21,2021-12-29 00:00:00+02,2022-01-02 00:00:00+02,Матохнюк Д.Б.,http://od.reyestr.court.gov.ua/files/51/0c9097...,1.0,2022-01-04 00:00:00+02
3,102391822,2670.0,5.0,4.0,40114.0,640/4558/19,2021-11-02 00:00:00+02,2022-01-02 00:00:00+02,Клименчук Н.М.,http://od.reyestr.court.gov.ua/files/51/904bf7...,1.0,2022-01-04 00:00:00+02
4,102391796,2670.0,3.0,4.0,40110.0,640/10043/21,2021-12-07 00:00:00+02,2022-01-02 00:00:00+02,Клименчук Н.М.,http://od.reyestr.court.gov.ua/files/51/4c9376...,1.0,2022-01-04 00:00:00+02
5,102391789,2670.0,3.0,4.0,40110.0,640/18672/21,2021-12-01 00:00:00+02,2022-01-02 00:00:00+02,Клименчук Н.М.,http://od.reyestr.court.gov.ua/files/51/f1fc47...,1.0,2022-01-04 00:00:00+02
6,102391769,2670.0,3.0,4.0,40088.0,640/3619/19,2021-11-10 00:00:00+02,2022-01-02 00:00:00+02,Клименчук Н.М.,http://od.reyestr.court.gov.ua/files/51/98d4ee...,1.0,2022-01-04 00:00:00+02
7,102391764,2670.0,3.0,4.0,40114.0,640/4230/19,2021-11-05 00:00:00+02,2022-01-02 00:00:00+02,Клименчук Н.М.,http://od.reyestr.court.gov.ua/files/51/dbaa6e...,1.0,2022-01-04 00:00:00+02
8,102391757,2670.0,3.0,4.0,40077.0,640/1726/20,2021-11-02 00:00:00+02,2022-01-02 00:00:00+02,Клименчук Н.М.,http://od.reyestr.court.gov.ua/files/51/145f06...,1.0,2022-01-04 00:00:00+02
9,102388442,1570.0,3.0,4.0,40074.0,420/20761/21,2021-12-30 00:00:00+02,2022-01-02 00:00:00+02,Потоцька Н.В.,http://od.reyestr.court.gov.ua/files/51/5514da...,1.0,2022-01-04 00:00:00+02
10,102390530,2670.0,5.0,4.0,40117.0,640/7590/19,2021-12-30 00:00:00+02,2022-01-02 00:00:00+02,Чудак О.М.,http://od.reyestr.court.gov.ua/files/51/654492...,1.0,2022-01-04 00:00:00+02
11,102390534,2670.0,5.0,4.0,40140.0,640/22201/18,2021-12-30 00:00:00+02,2022-01-02 00:00:00+02,Чудак О.М.,http://od.reyestr.court.gov.ua/files/51/e69098...,1.0,2022-01-04 00:00:00+02


Check how many criminal justice paper types:

In [10]:
administrative_document_data.shape

(72718, 12)

Discovering the criminal type of justise

In [11]:
criminal_document_data = document_data[document_data["justice_kind"] == 2]
criminal_document_data = criminal_document_data[criminal_document_data["doc_url"].notna()]
criminal_document_data.head(10)

,doc_id,court_code,judgment_code,justice_kind,category_code,cause_num,adjudication_date,receipt_date,judge,doc_url,status,date_publ
7591,102391625,1806.0,5.0,2.0,40455.0,592/1/22,2022-01-02 00:00:00+02,2022-01-02 00:00:00+02,Корольова Г. Ю.,http://od.reyestr.court.gov.ua/files/51/21be21...,1.0,2023-01-16 00:00:00+02
7592,102391694,2602.0,5.0,2.0,40455.0,753/25451/21,2022-01-02 00:00:00+02,2022-01-02 00:00:00+02,Котвицький В. Л.,http://od.reyestr.court.gov.ua/files/51/ac78a2...,1.0,2023-01-16 00:00:00+02
7593,102391699,1511.0,5.0,2.0,40456.0,501/1/22,2022-01-02 00:00:00+02,2022-01-02 00:00:00+02,Семенов О. А.,http://od.reyestr.court.gov.ua/files/51/a56209...,1.0,2023-01-16 00:00:00+02
7594,102391706,814.0,5.0,2.0,40455.0,334/1/22,2022-01-02 00:00:00+02,2022-01-02 00:00:00+02,Фетісов М. В.,http://od.reyestr.court.gov.ua/files/51/4e2053...,1.0,2023-01-16 00:00:00+02
7595,102391745,419.0,5.0,2.0,40455.0,204/1/22,2022-01-02 00:00:00+02,2022-01-02 00:00:00+02,Токар Н. В.,http://od.reyestr.court.gov.ua/files/51/1deff6...,1.0,2023-01-16 00:00:00+02
7596,102385278,2020.0,5.0,2.0,40455.0,641/1/22,2022-01-01 00:00:00+02,2022-01-02 00:00:00+02,Ященко С. О.,http://od.reyestr.court.gov.ua/files/51/8bac1f...,1.0,2023-01-16 00:00:00+02
7597,102391683,2316.0,5.0,2.0,40455.0,712/1/22,2022-01-02 00:00:00+02,2022-01-02 00:00:00+02,Борєйко О. М.,http://od.reyestr.court.gov.ua/files/51/b20ab7...,1.0,2023-01-16 00:00:00+02
7598,102391736,2506.0,5.0,2.0,40455.0,750/1/22,2022-01-02 00:00:00+02,2022-01-02 00:00:00+02,Кузнєцова О. О.,http://od.reyestr.court.gov.ua/files/51/9c7cbc...,1.0,2023-01-16 00:00:00+02
7599,102391695,2606.0,5.0,2.0,40452.0,757/2/22-к,2022-01-02 00:00:00+02,2022-01-02 00:00:00+02,Смик С. І.,http://od.reyestr.court.gov.ua/files/51/c98b87...,1.0,2023-01-16 00:00:00+02
7600,102391676,1622.0,5.0,2.0,40455.0,554/1/22,2022-01-02 00:00:00+02,2022-01-02 00:00:00+02,Тімошенко Н. В.,http://od.reyestr.court.gov.ua/files/51/b859e2...,1.0,2023-01-16 00:00:00+02


In [13]:
criminal_document_data[criminal_document_data["doc_id"] == 102383543]

,doc_id,court_code,judgment_code,justice_kind,category_code,cause_num,adjudication_date,receipt_date,judge,doc_url,status,date_publ
7965,102383543,1511.0,5.0,2.0,40455.0,501/5235/21,2021-12-31 00:00:00+02,2022-01-02 00:00:00+02,Тюмін Ю. О.,http://od.reyestr.court.gov.ua/files/51/023273...,1.0,2023-02-13 00:00:00+02


In [9]:
criminal_document_data[criminal_document_data["doc_id"] == 102394762]

,doc_id,court_code,judgment_code,justice_kind,category_code,cause_num,adjudication_date,receipt_date,judge,doc_url,status,date_publ
14437,102394762,604.0,1.0,2.0,40506.0,295/10306/21,2022-01-04 00:00:00+02,2022-01-04 00:00:00+02,Панченко Г. В.,http://od.reyestr.court.gov.ua/files/51/b8aa47...,1.0,2023-01-16 00:00:00+02


In [10]:
criminal_document_data[criminal_document_data["category_code"] == 40500].head(5)

,doc_id,court_code,judgment_code,justice_kind,category_code,cause_num,adjudication_date,receipt_date,judge,doc_url,status,date_publ
51758,102434525,2207.0,5.0,2.0,40500.0,675/15/22,2022-01-06 00:00:00+02,2022-01-06 00:00:00+02,Демчук П. В.,http://od.reyestr.court.gov.ua/files/51/979598...,1.0,2023-01-16 00:00:00+02
52351,102435357,2514.0,1.0,2.0,40500.0,740/5827/21,2022-01-06 00:00:00+02,2022-01-06 00:00:00+02,Олійник В. П.,http://od.reyestr.court.gov.ua/files/51/de46dc...,1.0,2023-01-16 00:00:00+02
52460,102429394,2515.0,5.0,2.0,40500.0,751/7/22,2022-01-05 00:00:00+02,2022-01-06 00:00:00+02,Карапута О. О.,http://od.reyestr.court.gov.ua/files/51/c24b2c...,1.0,2023-01-16 00:00:00+02
52528,102434854,1506.0,5.0,2.0,40500.0,496/4250/21,2022-01-06 00:00:00+02,2022-01-06 00:00:00+02,Буран В. М.,http://od.reyestr.court.gov.ua/files/51/597f31...,1.0,2023-01-16 00:00:00+02
52591,102439215,2408.0,5.0,2.0,40500.0,725/30/22,2022-01-05 00:00:00+02,2022-01-06 00:00:00+02,Іщенко І. В.,http://od.reyestr.court.gov.ua/files/51/031f93...,1.0,2023-01-16 00:00:00+02


In [11]:
criminal_document_data.shape

(87051, 12)

In [12]:
import requests

In [13]:
document_data["doc_url"].to_numpy()

array([nan,
       'http://od.reyestr.court.gov.ua/files/51/0c9097fb28ccabfcd4db8d140fd667b7.rtf',
       'http://od.reyestr.court.gov.ua/files/51/a53df6d681ad7a5921e67e15a6bce992.rtf',
       ...,
       'http://od.reyestr.court.gov.ua/files/51/86af8ada712fed7bb7327c9b55bb6b3c.rtf',
       'http://od.reyestr.court.gov.ua/files/51/86a45c24882ac2a836794b3cd0b91a9c.rtf',
       None], dtype=object)

In [14]:
document_data["doc_url"][1]

'http://od.reyestr.court.gov.ua/files/51/0c9097fb28ccabfcd4db8d140fd667b7.rtf'

In [15]:
requests.get(document_data["doc_url"][1])

<Response [200]>

### Uploading data to Cloud Storage

In [16]:
SUBFOLDER = "criminal_batch/"

In [17]:
storage_client = storage.Client()
bucket = storage_client.bucket("court_data_raw")

In [18]:
def download_court_file(url: str, doc_id: str):
    rtf_court_raw_filename = url.split("/")[-1]
    result_filename = SUBFOLDER + str(doc_id) + ".rtf"
    blob = bucket.blob(result_filename)
    
    response = requests.get(url)
    # print(f"response -> {response.content}")
    response.raise_for_status()
    
    blob.upload_from_string(response.content, content_type=response.headers.get('content-type'))

#### Uploading 50 instances

In [19]:
counter = 20000

In [20]:
for index, row in criminal_document_data[20000:30000].iterrows():
    if counter % 1000 == 0:
        print(f"Processed -> {counter}")
            
    if pd.isna(document_data.at[index, "doc_url"]):
        continue
        
    try:
        download_court_file(
            document_data["doc_url"][index], 
            document_data["doc_id"][index],
        )
    except:
        print("Unable to download court data -> ", document_data["doc_url"][index], document_data["doc_id"][index])
    
    counter = counter + 1

Processed -> 20000
Unable to download court data ->  http://od.reyestr.court.gov.ua/files/51/b3c6f4b6f0f6485c8fbf899c87083ebb.rtf 102531137
Unable to download court data ->  http://od.reyestr.court.gov.ua/files/51/b3160f09ddc2e35f20390d51e37724ea.rtf 102541679
Unable to download court data ->  http://od.reyestr.court.gov.ua/files/51/a6a63428d6e8a392f8b3925ed39d432c.rtf 102537012
Unable to download court data ->  http://od.reyestr.court.gov.ua/files/51/f809a3c94f8b7d5dccb4d23c1ae2c2fe.rtf 102541654
Unable to download court data ->  http://od.reyestr.court.gov.ua/files/51/e9fefb5dc8f48cbf82b56bba744b70e9.rtf 102531057
Unable to download court data ->  http://od.reyestr.court.gov.ua/files/51/105b3cd92d12177b01f0d2767eb6438c.rtf 102531591
Unable to download court data ->  http://od.reyestr.court.gov.ua/files/51/f46fa987b2911d9f3c855a0f9600df2c.rtf 102541704
Unable to download court data ->  http://od.reyestr.court.gov.ua/files/51/fbfcb7b04225ed54b73b5808488224f8.rtf 102541632
Unable to dow

In [62]:
print("Hello")

Hello
